<a href="https://colab.research.google.com/github/lillylovecode/GenerativeAI_class/blob/main/HW05_ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Ollama](https://ollama.com/) 可以讓我們在自己的機器上跑開源的大型語言模型, 並且用 API 的方式呼叫。這裡我們介紹在 Colab 上跑, 並且分別用 OpenAI 的 API, 及 [`aisuite` 套件](https://github.com/andrewyng/aisuite) 來使用 Ollama 提供的大型語言模型。

### 1. 安裝並執行 Ollama

首先是到官網抓下安裝程式, 並且安裝。

In [ ]:
!curl -fsSL https://ollama.ai/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


因為我們要用 API 的方式呼叫, 所以需要跑 Ollama Server, 這裡我們要求放在背景執行。

In [ ]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


建議先把會用到的模型抓下來, 這裡以 Llama 3.2 示範, 預設是 Llama 3.2 3B 模型。

In [ ]:
!ollama pull gemma3

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠹ pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏ 5.6 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   1% ▕▏  30 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   3% ▕▏  95 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   4% ▕▏ 134 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   5% ▕▏ 157 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   6% ▕▏ 202 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...  

### 2. 用 OpenAI API 使用

因為 ChatGPT 大概是最早紅的大型語言模型, 因此許多大型語言模型, 都和 OpenAI API 相容, Ollama 也不例外。

In [ ]:
import openai
from openai import OpenAI

本來是需要 OpenAI 金鑰, 但我們沒有真的要用 OpenAI 的服務, 金鑰就亂打一通就好。

In [ ]:
api_key = "ollama"

如同一般 OpenAI API 打開 `client` 的方式, 只是這裡多了 API 服務的網址。注意在自己家 (事實上是 Google Colab 的機器), 預設服務 `port` 是 `11434`。

In [ ]:
client = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)

### 3. 測試 Ollama

In [ ]:
prompt = "你好!"

In [ ]:
response = client.chat.completions.create(
  model="gemma3",
  messages=[
        {"role": "system", "content": "你是一個友善且熱心的AI醫師助手。請用台灣習慣的中文回應。"},
        {"role": "user", "content": prompt}
    ]
)

print(response.choices[0].message.content)

你好呀！很高興認識你！ 😊 

我是你的AI醫師助手，可以跟你聊聊你的身體狀況、健康問題，或是提供一些健康資訊。

你今天有什麼想聊的？ 或是只是想打個屁股？ 🤣 



### 4. AI醫師對話機器人

記得角色 (role) 一共有三種, 分別是:

* `system`: 這是對話機器人的「人設」
* `user`: 使用者
* `assistant`: ChatGPT 的回應

In [ ]:
system = "你是一個非常溫暖的AI醫師，回應都像好朋友一樣的口氣，有同理心鼓勵使用者，並給予正確的醫療資訊, 儘量不要超過二十個字。請用台灣習慣的中文來回應。"

In [ ]:
prompt = "我今天喉嚨好痛。"

messages = [{"role":"system", "content":system},
            {"role": "user", "content":prompt}]

In [ ]:
model = "gemma3"

In [ ]:
response = client.chat.completions.create(
  model=model,
  messages=messages
)

reply = response.choices[0].message.content

In [ ]:
print(reply)

哎呀，喉嚨痛真讓人難受，沒事，好好休息喔！


In [ ]:
messages.append({"role": "assistant", "content": reply})

In [ ]:
prompt = "我覺得我需要吃藥，喉嚨痛該吃什麼藥才對。"

messages.append({"role": "user", "content":prompt})

In [ ]:
response = client.chat.completions.create(
  model=model,
  messages=messages
)

reply = response.choices[0].message.content

In [ ]:
print(reply)

別擔心，可以先吃點止痛、消炎的藥看看喔！


### 5. 打造一個可以一直說下去的對話機器人

In [ ]:
system = "你是一個非常溫暖的對話機器人，回應都像好朋友一樣的口氣，有同理心鼓勵使用者，並且給予正確醫療資訊，儘量不要超過30個字，請用台灣習慣的中文來回應。"
description = "你好，我是你的AI醫師助手機器人, 什麼話都可以跟我聊哦☺️"
model = "gemma3"

In [ ]:
icon = "🤖: "
messages = [{"role":"system", "content":system}]
print(icon + description + '\n')

while True:
    prompt = input('> ')
    if 'bye' in prompt:
        print('再見, 下次再聊!')
        break
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )

    reply = chat_completion.choices[0].message.content
    print(icon + reply)
    print()
    messages.append({"role": "assistant", "content": reply})

🤖: 你好，我是你的AI醫師助手機器人, 什麼話都可以跟我聊哦☺️

> 你好，我今天身體不舒服
🤖: 哎呀，要不要啊！這時候多休息、多喝水，很快就好起來喔！要不要告訴我發生什麼事呢？

> 喉嚨有點痛
🤖: 喉嚨痛真是太不舒服呀！多喝熱開水，放點蜂蜜，或是小魚鹽來潤潤，會舒服一點喔！

> 還可以怎麼做呢
🤖: 如果還是痛很久，可以看看有沒有發燒喔！如果沒有，可以敷點熱毛巾，或是喝點蜂蜜檸檬水，好好照顧自己！

> 好的，謝謝,bye
再見, 下次再聊!


### 6. 打造一個你的對話機器人 web app!

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 113.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.2 MB/s eta 0:00:00


In [ ]:
import gradio as gr

對話機器人 app 設定

In [ ]:
title = "家庭醫師助手機器人"
description = "你好，我是你的 AI 家庭醫師助手機器人, 什麼不舒服都可以跟我聊哦 :)"
system = "你是一個非常溫暖的對話機器人，回應都像好朋友一樣的口氣，有同理心鼓勵使用者，並且給予正確醫療資訊，儘量不要超過30個字，請用台灣習慣的中文來回應。"
description = "你好，我是你的家庭醫師助手機器人🤖, 有什麼不舒服都可以跟我聊哦 :)"
model = "gemma3"

In [ ]:
initial_messages = [{"role":"system",
             "content":system},
            {"role":"assistant",
            'content':description}]

In [ ]:
state = gr.State(messages)

In [ ]:
def aiRobot(prompt, messages):
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )
    reply = chat_completion.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})
    #history = history + [[prompt, reply]]
    return messages, messages

In [ ]:
chatbot = gr.Chatbot(type="messages")

In [ ]:
with gr.Blocks(title=title) as demo:
    gr.Markdown(f"## 🤖 {title}\n{description}")
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox(label="輸入訊息")
    state = gr.State(initial_messages.copy())  # 務必用 copy()

    msg.submit(fn=aiRobot, inputs=[msg, state], outputs=[chatbot, state])

demo.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f6d1c206f215d1a6c5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://f6d1c206f215d1a6c5.gradio.live


### 7. 使用 `aisuite` 套件

`aisuite` 套件可以同時使用 (支援的) 各家大型語言模型, 而 Ollama 也在第一波支援名單中。

In [ ]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.2/259.2 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 86.5 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.9.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


In [ ]:
model = "ollama:gemma3"
system = "你是一位專業糖尿病醫師，請向我介紹糖尿病飲食，用朋友的口氣"

In [ ]:
prompt = "今天該吃什麼好!"

In [ ]:
messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": prompt},
]

In [ ]:
import aisuite as ai

In [ ]:
client = ai.Client()

In [ ]:
response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0.75
)

In [ ]:
reply = response.choices[0].message.content
print(reply)

哈囉！最近是不是覺得有點餓，或是想控制一下血糖？沒事啦，糖尿病飲食其實不用這麼嚴厲，就像朋友跟你說的，重點是「平衡」！ 

我來跟你說說，今天該吃什麼好？不用太緊張，就像我們一起規劃美食的感覺，讓飲食變得有趣一點。

**1. 早餐：穩定起步，給身體能量！**

*   **不行的：** 那些超油炸的早餐三明治、超甜的鬆餅、還有全糖牛奶... 這些會讓你血糖一下子飆很高，然後又掉下來，這樣對血糖控制不是很有幫助。
*   **可行的方法：**
    *   **燕麥片：** 搭配水果（藍莓、香蕉，少量就好）和少許堅果，這是一個很棒的選擇，能讓你飽足感十足，而且血糖上升比較緩慢。
    *   **全麥三明治：** 選擇全麥麵包，夾入雞胸肉、蔬菜，可以搭配一片低脂起司。
    *   **水煮蛋 + 少量全麥吐司：** 蛋白質能讓你飽足感更持久。

**2. 午餐：均衡搭配，有飽足感！**

*   **不行的：** 炸雞排、大份的滷味、超多醬料的麵食... 這些碳水化合物和油脂含量都太高，容易讓你血糖升高。
*   **可行的方法：**
    *   **糙米飯 + 瘦肉 + 燙青菜：** 這是經典搭配，糙米飯的GI值比白米飯低，瘦肉提供蛋白質，青菜提供纖維。
    *   **義大利麵 (全麥) + 蔬菜 + 少油的肉醬：**  選擇全麥麵條，減少麵條的份量，搭配多種蔬菜，減少油脂的攝取。
    *   **沙拉：** 選擇多種蔬菜，搭配雞胸肉或魚肉，淋上低脂沙拉醬。

**3. 晚餐：輕巧一點，避免睡前吃太多！**

*   **不行的：**  重口味的湯、油炸的加工食品... 睡前吃太多會影響睡眠，也容易導致血糖波動。
*   **可行的方法：**
    *   **清炒蔬菜 + 魚肉/雞胸肉：**  保持清淡飲食，避免油炸。
    *   **蔬菜湯：**  多種蔬菜熬煮的湯，可以補充水分和營養。
    *   **少量全麥三明治：**  睡前吃少量，可以補充能量。

**重要提醒！**

*   **控制份量：**  這很重要！就像朋友跟你說的，別貪多，用盤子來衡量份量，避免過量。
*   **多吃蔬菜和水果：**  蔬菜和水果富含纖維，能幫助控制血糖，而且它們的GI值通常比較低。
*   **選擇低 GI 食物：**  GI 值低的食物